In [1]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta
from connection_keys import connect_bq_client
import statsmodels.api as sm
import os
import warnings
import itertools
warnings.filterwarnings("ignore")

In [23]:
client = connect_bq_client()

sql = """
    SELECT *
    FROM `db_market_product.gpu`
    ORDER BY dt_carga asc
"""

df = client.query(sql).to_dataframe()

In [24]:
df['data'] = df.dt_carga.dt.date
df.head()

,ds_marca,ds_nome_produto,vl_preco_atual,vl_preco_com_desconto,vl_porcentagem_do_desconto,qt_quantidade_estoque,oferta,ds_disponibilidade,loja,dt_carga,data
0,Nvidia,Nvlink 3 Slot Rtx Geforce,1322.0,1322.0,0.0,0,None,False,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29
1,Gigabyte,"Gigabyte GTX 1050 Ti, 4GB, DDR5",1999.0,1999.0,0.0,0,None,False,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29
2,Nvidia,Vga Pci-e 1gb Galax Geforce Gt710 - 1gb,339.0,339.0,0.0,0,None,False,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29
3,Colorful,"Placa de Video Colorful GT 710, 1GB DDR3",349.0,349.0,0.0,93,None,True,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29
4,Asus,Placa de Vídeo Asus GeForce RTX 3080 10GB,19999.0,19999.0,0.0,0,None,False,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359531 entries, 0 to 359530
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   ds_marca                    359531 non-null  object             
 1   ds_nome_produto             359531 non-null  object             
 2   vl_preco_atual              359531 non-null  float64            
 3   vl_preco_com_desconto       359531 non-null  float64            
 4   vl_porcentagem_do_desconto  359531 non-null  float64            
 5   qt_quantidade_estoque       133662 non-null  Int64              
 6   oferta                      359531 non-null  object             
 7   ds_disponibilidade          359531 non-null  object             
 8   loja                        359531 non-null  object             
 9   dt_carga                    359531 non-null  datetime64[ns, UTC]
 10  data                        359531 non-null 

In [27]:
df.groupby(['dt_carga'])['loja'].count()
df.groupby(['data', 'loja'])['dt_carga'].nunique()
# df['data'].value_counts().sort_index().reset_index()
# df['data'] = df.dt_carga.dt.date

data        loja  
2022-09-29  Kabum     2
            Pichau    2
2022-09-30  Kabum     2
            Pichau    2
2022-10-01  Kabum     2
                     ..
2022-11-29  Pichau    2
2022-11-30  Kabum     2
            Pichau    2
2022-12-01  Kabum     2
            Pichau    2
Name: dt_carga, Length: 128, dtype: int64

In [28]:
# df_datas_loja = df.groupby(['data', 'loja'])['dt_carga'].nunique()
# df_datas_loja
maxDf = df.groupby(['data'])['dt_carga'].max()
maxDates = maxDf.tolist()
df.loc[df['dt_carga'].isin(maxDates)]

,ds_marca,ds_nome_produto,vl_preco_atual,vl_preco_com_desconto,vl_porcentagem_do_desconto,qt_quantidade_estoque,oferta,ds_disponibilidade,loja,dt_carga,data
2663,Nvidia,Nvlink 3 Slot Rtx Geforce,1322.00,1322.00,0.0,0,None,False,Kabum,2022-09-29 22:01:30.406181+00:00,2022-09-29
2664,Gigabyte,"Gigabyte GTX 1050 Ti, 4GB, DDR5",1999.00,1999.00,0.0,0,None,False,Kabum,2022-09-29 22:01:30.406181+00:00,2022-09-29
2665,Nvidia,Vga Pci-e 1gb Galax Geforce Gt710 - 1gb,339.00,339.00,0.0,0,None,False,Kabum,2022-09-29 22:01:30.406181+00:00,2022-09-29
2666,Colorful,"Placa de Video Colorful GT 710, 1GB DDR3",349.00,349.00,0.0,93,None,True,Kabum,2022-09-29 22:01:30.406181+00:00,2022-09-29
2667,Asus,Placa de Vídeo Asus GeForce RTX 3080 10GB,19999.00,19999.00,0.0,0,None,False,Kabum,2022-09-29 22:01:30.406181+00:00,2022-09-29
...,...,...,...,...,...,...,...,...,...,...,...
359526,Power Color,"Placa de Video Power Color Radeon AXRX 6400, 4...",1176.46,999.99,15.0,97,"[{'ds_oferta_nome': 'Natal Ninja', 'vl_oferta_...",True,Kabum,2022-12-01 22:01:39.701577+00:00,2022-12-01
359527,Power Color,Placa de Vídeo RX 6500 XT ITX Power Color AMD ...,1529.40,1299.99,15.0,0,"[{'ds_oferta_nome': 'Prime', 'vl_oferta_preco_...",False,Kabum,2022-12-01 22:01:39.701577+00:00,2022-12-01
359528,Power Color,Placa de Vídeo RX 6800 XT PowerColor AMD Radeo...,5882.34,4999.99,15.0,0,"[{'ds_oferta_nome': 'Prime', 'vl_oferta_preco_...",False,Kabum,2022-12-01 22:01:39.701577+00:00,2022-12-01
359529,Power Color,Placa de Vídeo AXRX 6600 3DH Power Color Fight...,3999.99,3399.99,15.0,0,"[{'ds_oferta_nome': 'Prime', 'vl_oferta_preco_...",False,Kabum,2022-12-01 22:01:39.701577+00:00,2022-12-01


In [71]:
df_datas_loja = df_datas_loja.to_frame(name='contagem')
df_datas_loja_unstack = df_datas_loja.unstack('loja')
# df_datas_loja_unstack.columns
# df_datas_loja_unstack_teste = df_datas_loja_unstack.columns.set_levels(['data','kabum','pichau'],level=1)
d = dict(zip(df.columns.levels[1], ))
# df_datas_loja_unstack_teste

AttributeError: 'Index' object has no attribute 'levels'

In [13]:
df.loc[df['data'].astype(str) == '2022-10-01']['dt_carga'].unique()
# df

<DatetimeArray>
['2022-10-01 10:02:27.606722+00:00', '2022-10-01 22:05:17.603092+00:00',
 '2022-10-01 22:07:59.247261+00:00']
Length: 3, dtype: datetime64[ns, UTC]

In [9]:
type(df['data'][0])

datetime.date